In [0]:
import zipfile
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive, files
from PIL import Image
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation,Flatten
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.preprocessing.image import array_to_img
from keras.utils import np_utils
%matplotlib inline

drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


## Unzipping the file with the images

In [0]:
with zipfile.ZipFile('/content/gdrive/My Drive/Big Data/Projet/celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/img/')

# Création de la base de données

In [0]:
samp = pickle.load(open('/content/gdrive/My Drive/Big Data/Projet/Listes sample/Pale_Skin.pkl', 'rb'))

In [0]:
path = '/content/img/img_align_celeba/img_align_celeba/'
target = []
data = np.zeros((len(samp), 64, 64, 3)) #dimension of the images
count = 0
for file in os.listdir(path) :
  if file in samp :
    image = Image.open(path + file)
    image = image.resize((64, 64))
    data[count] = np.array(image)
    target.append(file)
    count += 1

In [0]:
data.shape

(18701, 64, 64, 3)

In [0]:
all_attributes = pd.read_csv('/content/img/list_attr_celeba.csv')
pale_skin = all_attributes[['image_id','Pale_Skin']]
pale_skin = pale_skin.loc[pale_skin[pale_skin['image_id'].isin(target) == True].index]
pale_skin = pale_skin.set_index('image_id')
pale_skin = pale_skin.loc[target]
pale_skin.head()

,Pale_Skin
image_id,
060363.jpg,-1
075566.jpg,1
104432.jpg,-1
079233.jpg,1
097429.jpg,-1


# Creation of the train and test sets

In [0]:
y = np.array(pale_skin.Pale_Skin)
y[y==-1] = 0

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.2)

In [0]:
print(X_train.shape, y_train.shape)

(14960, 64, 64, 3) (14960,)


# Creation of the NN

In [0]:
pale_skin_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 4, 4, 350)         403550    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 1, 1, 350)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 350)              

In [0]:
conv_net = Sequential()

# convolution layer 1
conv_net.add(Conv2D(64, (3, 3), activation = 'relu', input_shape = (64, 64, 3)))
conv_net.add(MaxPooling2D(pool_size = (3, 3)))
#conv_net.add(Dropout(0.5))


# convolution layer 2
conv_net.add(Conv2D(128, (3, 3), activation = 'relu'))
conv_net.add(MaxPooling2D(pool_size = (3, 3)))


conv_net.add(Conv2D(350, (3, 3), activation = 'relu'))
conv_net.add(MaxPooling2D(pool_size = (3, 3)))

# fully connected
conv_net.add(Flatten())
# conv_net.add(Dense(1028, activation='relu'))
# conv_net.add(Dense(256, activation='relu'))
# conv_net.add(Dropout(0.5))
conv_net.add(Dense(128, activation = 'relu'))
conv_net.add(Dropout(0.5))
conv_net.add(Dense(64, activation = 'relu'))
conv_net.add(Dense(1, activation = 'sigmoid'))

conv_net.compile(loss = 'binary_crossentropy', optimizer = 'adam', 
                 metrics = ['accuracy'])

## callbacks to save the best model

In [0]:
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2,
                              patience = 1, min_lr = 0.001)

# Path where we will save our model
filepath = "/content/gdrive/My Drive/Big Data/models pale_skin/PaleSkin.hdf5" 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy',
                             verbose = 1, save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, reduce_lr]

In [0]:
history = conv_net.fit(X_train, y_train, batch_size = 128, epochs = 20,
                       validation_data = (X_test, y_test), callbacks = callbacks) 
#total of 30 epochs

Epoch 1/20
117/117 [==============================] - ETA: 0s - loss: 0.1388 - accuracy: 0.9457
Epoch 00001: val_accuracy did not improve from 0.88853
117/117 [==============================] - 62s 534ms/step - loss: 0.1388 - accuracy: 0.9457 - val_loss: 0.3791 - val_accuracy: 0.8778 - lr: 0.0010
Epoch 2/20
117/117 [==============================] - ETA: 0s - loss: 0.1503 - accuracy: 0.9382
Epoch 00002: val_accuracy did not improve from 0.88853
117/117 [==============================] - 62s 531ms/step - loss: 0.1503 - accuracy: 0.9382 - val_loss: 0.3404 - val_accuracy: 0.8867 - lr: 0.0010
Epoch 3/20
117/117 [==============================] - ETA: 0s - loss: 0.1321 - accuracy: 0.9479
Epoch 00003: val_accuracy did not improve from 0.88853
117/117 [==============================] - 62s 529ms/step - loss: 0.1321 - accuracy: 0.9479 - val_loss: 0.4056 - val_accuracy: 0.8439 - lr: 0.0010
Epoch 4/20
117/117 [==============================] - ETA: 0s - loss: 0.1286 - accuracy: 0.9470
Epoch 0000

## Load the saved model

In [0]:
pale_skin_model = load_model("/content/gdrive/My Drive/Big Data/models pale_skin/PaleSkin.hdf5")

In [0]:
accuracy = pale_skin_model.evaluate(X_test, y_test, verbose=0)[1]
print(accuracy)

0.8885324597358704
